# Comp 472 Mini-Project 2

## 1. Game Setup

In [1]:
import pprint
from queue import PriorityQueue
import copy
import time
from pandas import DataFrame

In [2]:
# Definition of car object 
class Car:
    def __init__(self, name, fuel, coordinates, orientation):
        
        self.name = name
        self.fuel = fuel
        self.coordinates = coordinates # list of coordinates that represents its position in the board
        self.orientation = orientation
        
    # Function used to print the information of the car
    def printCarInfo(self):
        print("Name: ", self.name, ", Fuel: ",self.fuel ,", Coordinates: ", self.coordinates, ", Orientation: ", self.orientation)

In [3]:
# Possible letters that could represent cars in the grid
carLetters =   ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

# Definition of board object
class Board:
    
    # initializer
    
    def __init__(self, puzzleLine): # default cost = 0, for successors do parentboard.cost +1
        self.dimension = 6 # dimension
        
        
        
        self.parent = None 
        self.grid = [] # 
        self.cars = {} # dictionary of car objects present in the board  {carName: car object}
        self.puzzleLine = puzzleLine # string 

        # f = g + h
        self.cost = 0
        self.heuristic = 0
        self.f = 0
        
        self.successorMove =""   # "M down 1 "
        
        
        
        
        # set initial info  ---------------------TODO: is it okay to not pass param?
        self.fillGrid()
        coordinatesDict = self.setCoordinate()
        fuelDict = self.setInitialFuel()
        self.createCarObjects(coordinatesDict,fuelDict)
                

    def setCost(self, cost):
        self.cost = cost
        
    def setHeuristic(self,heuristic):
        self.heuristic = heuristic
    
    def setF(self,cost,heuristic):
        self.f = cost + heuristic
        
    def getCost(self):
        return self.cost
    
    def getHeuristic(self):
        return self.heuristic
    
    def getF(self):
        return self.f
            
            
            
            
    def fillGrid(self):
        # Creating empty grid
        self.grid = [[0 for x in range(self.dimension)] for x in range(self.dimension)] 

        # Filling grid with puzzle line information (36 characters)
        a = 0    
        for i in range(self.dimension):
            for j in range(self.dimension):
                self.grid[i][j] = self.puzzleLine[a] # walk through up until 6*6 =36 characters
                a += 1

    def setCoordinate(self):
         # Getting coordinates of the cars in the grid
        coordinatesDict = {}
        for letter in carLetters:
            if letter in self.puzzleLine: # If car letter exists in the board
                coordinatesDict[letter] = [[x, y] for x, li in enumerate(self.grid) for y, val in enumerate(li) if val==letter]
        return coordinatesDict
                    
    def setInitialFuel(self):
         # Checking for initial fuel units for the cars in the board \for the specific puzzle (Default fuel units = 100)
        fuelDict = {}
        fuelInfoFromPuzzleLine = self.puzzleLine[(self.dimension*self.dimension):].strip()
        if fuelInfoFromPuzzleLine != "": # If additional info exists after the puzzle line, then initial fuel info exists
            initialFuelInfo = fuelInfoFromPuzzleLine.split()
            for fuel in initialFuelInfo:
                fuelDict[fuel[0]] = int(fuel[1:])
        return fuelDict

    
    def createCarObjects(self,coordinatesDict,fuelDict):
                # Creating car object with obtained information
        for carName in coordinatesDict:

            # Checking the orientation of each car in the board
            # If the x of the coordinates are equal, then horizontal. If not, then vertical
            if coordinatesDict[carName][0][0] == coordinatesDict[carName][1][0]:
                orientation = "horizontal"
            else:
                orientation = "vertical"

            if carName in fuelDict:
                self.cars[carName] = Car(carName, fuelDict[carName], coordinatesDict[carName], orientation)
            else:
                self.cars[carName] = Car(carName, 100, coordinatesDict[carName], orientation)
     
                
    # Function to print the initial Grid from string       
    def returnPuzzleLine(self):
        grid = ""
        a = 0
        for i in range(self.dimension):
            grid = grid+ self.puzzleLine[a:(a+6)]+"\n"
            a += 6
        return grid
              
    
    # Function to print the current board
    def printGrid(self):
        pprint.pprint(self.grid)
        
        

    def getPuzzleLine(self):
        successorStr = ""
        for line in self.grid:
            for char in line:
                successorStr += char

        for car in self.cars.values():
            if car.fuel < 100:
                successorStr += (" " + car.name + str(car.fuel)) 
        return successorStr


    # Function to generate the possible states that the board can be in depending on the possible moves of one car    
    def generateSuccessorBoards(self):
        successors = [] # List of successor objects (each contains info needed for the moveCar function)

        
        for car in self.cars.values():
            # horizontal -- ======================
            if car.orientation == "horizontal":

                positionsMoved = 0
                # Check left direction ===========
                left = car.coordinates[0][:] # copy most left coordinate of this car
                while(left[1] > 0): # until we reach left edge of board
                    left[1] -= 1 # check one cell to the left
                    
                    if self.grid[left[0]][left[1]] == '.': # if that cell is empty
                        positionsMoved += 1 
                        # check if enough fuel?
                        if (car.fuel) - positionsMoved < 0:
                            # don't genrerate successors? 
                            continue
                        else:
                            newCoordinates = [row[:] for row in car.coordinates]
                            # [[x,y],[z,t]]
                            for each in newCoordinates:
                                each[1] -= positionsMoved
                            
                            # create one child
                            child = self.createOneSuccessor(newCoordinates, positionsMoved, car.name, car.coordinates,"left" )
                               
                            # append to successors list <- contains child objects
                            successors.append(child)
         
                        
                    else:
                        break

                # Check right direction ===========
                
                positionsMoved = 0
                right = car.coordinates[-1][:] # copy most right coordinate of this car

                while(right[1] < 5): # until we reach right edge of board
                    right[1] += 1 # check one cell to right

                    if self.grid[right[0]][right[1]] == '.':
                        
                        positionsMoved += 1 
                        # check if enough fuel?
                        if (car.fuel) - positionsMoved < 0:
                            # don't genrerate successors? 
                            continue
                        else:
                            newCoordinates = [row[:] for row in car.coordinates]
                            # [[x,y],[z,t]]
                            for each in newCoordinates:
                                each[1] += positionsMoved
                            
                            # create one child
                            child = self.createOneSuccessor(newCoordinates, positionsMoved, car.name, car.coordinates ,"right")
                                
                            # append to successors list <- contains child objects
                            successors.append(child)
         
                    else:
                        break

            # vertical | ======================
            if car.orientation == "vertical":
                
                # Check up direction ===========
                positionsMoved = 0
                up = car.coordinates[0][:]
                
                while(up[0] > 0):
                    up[0] -= 1
                    if self.grid[up[0]][up[1]] == '.':
                        positionsMoved += 1 
                        # check if enough fuel?
                        if (car.fuel) - positionsMoved < 0:
                            # don't genrerate successors? 
                            continue
                        else:
                            newCoordinates = [row[:] for row in car.coordinates]
                            # [[x,y],[z,t]]
                            for each in newCoordinates:
                                each[0] -= positionsMoved
                            
                            # create one child
                            child = self.createOneSuccessor(newCoordinates, positionsMoved, car.name, car.coordinates,"up" )
                            
                           
                            
                                
                            # append to successors list <- contains child objects
                            successors.append(child)
         
                    else:
                        break


                # Check down direction ===========
                
                positionsMoved = 0
                down = car.coordinates[-1][:]
                
                while(down[0] < 5):
                    down[0] += 1
                    if self.grid[down[0]][down[1]] == '.':
                        positionsMoved += 1 
                        # check if enough fuel?
                        if (car.fuel) - positionsMoved < 0:
                            # don't genrerate successors? 
                            continue
                        else:
                            newCoordinates = [row[:] for row in car.coordinates]
                            # [[x,y],[z,t]]
                            for each in newCoordinates:
                                each[0] += positionsMoved
                            
                            # create one child
                            child = self.createOneSuccessor(newCoordinates, positionsMoved, car.name, car.coordinates, "down" )
                            # append to successors list <- contains child objects
                            successors.append(child)
         
                    else:
                        break

        
        return successors 
    
    
    
    # helper function (used inside updateBoardAndCarInfo function)
    def updateGrid(self, carName, oldCoordinates, newCoordinates ):
        
        for each in oldCoordinates:
            self.grid[each[0]][each[1]] = '.'
        for each in newCoordinates:
            self.grid[each[0]][each[1]] = carName


    
    def createOneSuccessor(self, newCoordinates, positionsMoved, carName, oldCoordinates, direction ):
        child = copy.deepcopy(self) # deepcopy of current board
        child.parent = self
        child.cars[carName].coordinates = newCoordinates # update car coordinate
        child.updateGrid(carName, oldCoordinates, newCoordinates ) # update gird
        child.cars[carName].fuel -= positionsMoved # update fuel
        child.cost = self.cost + 1 
        child.puzzleLine = child.getPuzzleLine()
        child.successorMove = carName+" "+direction+" "+str(positionsMoved)+"    "+str(child.cars[carName].fuel)
        
        return child
    
    
 
    # Function that checks if the car is A    
    def isGoal(self):

        
        Ambulance = self.cars['A']
        if Ambulance.coordinates[-1] == [2,5]:
            return True
        else:
            return False
    
    # Function that checks if the car is at the exit
    def leaveGridIfAtExit(self):
        
        
        for car in (self.cars).values():
        

            # coordinates should also be updated
            # remove car from coordinatesDict
            if car.orientation == "horizontal":
                if car.coordinates[-1] == [2,5]:
                    # exit
                    for each in car.coordinates:
                        self.grid[each[0]][each[1]] = '.'   
                    (self.cars).pop(car.name)
                break
            else:
                pass

    
    
    
    def printBoardInfo(self):
        print(self.puzzleLine)
        print("Board: ")
        self.printGrid()
        for each in (self.cars).values():
            each.printCarInfo()

## File 

In [4]:
file_path = "SampleInputOutput/Sample/sample-input.txt"

def readPuzzles(file_path):
    hashtag = "#"
    with open(file_path) as file:
        puzzles = [line.rstrip() for line in file]
        puzzles = list(filter(None, puzzles))

        for line in puzzles.copy():
            if hashtag in line:
                puzzles.remove(line)
    return puzzles

In [5]:
# Boards = []
# def printAllInfo(file_path):
#     a=1
#     lines = readPuzzles(file_path) # extract lines from input file
    
    
    
#     for each in lines: # for each board, print infos
#         print("--Board ",a,"--")
#         test = Board(each)
#         Boards.append(test)
#         test.printInitialBoard()
#         print("\n>>Successors: ")
#         pprint.pprint(test.generateSuccessorStates())
#         print("\n>>Car info")
#         for car in test.cars.values():
#             car.carInfo()
#         print("==================================\n")
#         a+=1

## 2.1 State Space Search

### 2.1.1 Uniform Cost Search

In [6]:
# helper function used in all three search
def getSolutionPath(CLOSED, Root):
    solutionPath= []
    solutionPathStr = ""
    
    if len(CLOSED) == 1:
        return solutionPath
        
    
    goalState =  CLOSED[-1] # last element is our goal state
    
    while True:
        if goalState == Root:
            solutionPath.reverse()
            solutionPath.append(solutionPathStr)
            return solutionPath
        else:
            solutionPathStr = solutionPathStr + goalState.successorMove+"; "
            solutionPath.append(goalState.successorMove+" "+goalState.puzzleLine)
            goalState = goalState.parent
    
        
        

In [7]:
def UniformCostSearch(Root):
    CLOSED = []
    OPEN = []
    
    
    
    
    
    # Append the rootState (initial state) to the OPEN list
    OPEN.append(Root)
    
    # Check if the rootState is goal
    if Root.isGoal():
        OPEN.pop(0)
        CLOSED.append(Root)
        return CLOSED, getSolutionPath(CLOSED, Root)
    
    while True:
        
        # If the OPEN list is empty, then there is no solution found
        if not OPEN:
#             print("No Solution found.")
            return CLOSED, getSolutionPath(CLOSED, Root)
        
        
        
        # "Visit" the first board in the OPEN list and add it to the CLOSED list
        parentBoard = OPEN.pop(0)
        CLOSED.append(parentBoard)

        
        # Checking if the current board we are visiting is the goal state
        if  parentBoard.isGoal():
            # TO-DO!!: AFTER FINDING THE GOAL STATE WE HAVE TO BACKTRACK AND FIND SOLUTION PATH
            
            
            

            # CURRENTLY ONLY TEMPORARILY PRINTING OPEN CLOSED LISTS
#             print("CLOSED List: ")
#             for board in CLOSED:
# #                 print("\n")
# #                 board.printBoardInfo()
#                 print(board.puzzleLine)
   
            
#             print("OPEN List: ")
#             for board in OPEN:
# #                 print("\n")
# #                 board.printBoardInfo()
#                 print(board.puzzleLine)
            
            return CLOSED, getSolutionPath(CLOSED, Root)
        else: 
            parentBoard.leaveGridIfAtExit()
            
            
            
        # Generate possible successor moves for the cars on the board
        successors = parentBoard.generateSuccessorBoards()
        
        
        
        for child in successors:
            
               
            # Checking if the current successor board is already present in the CLOSED list
            if(not any(child.grid == visitedBoard.grid for visitedBoard in CLOSED)):
                # If the OPEN list is empty, we will append the first child
                if not OPEN:
                    OPEN.append(child)
                else:    
                    # Checking if the current successor board is already present in the OPEN list
                    # If it is and the cost of its "clone" is greater, then we will replace it with the current successor
                    if any((child.grid == toBeVisited.grid and child.cost < toBeVisited.cost) for toBeVisited in OPEN):
                        OPEN.remove(toBeVisited)
                        OPEN.append(child)
                     # If it is a new successor, we append it to the list   
                    elif any((child.grid == toBeVisited.grid) for toBeVisited in OPEN):
                        continue
                    else:
                        OPEN.append(child)
        
        # Sort this list by cost (priority by cost)
        OPEN.sort(key=lambda x: x.cost, reverse=False)


In [8]:
def GreedyBestFirstSearch(Root, heuristic):
    CLOSED = []
    OPEN = []
    
    
    
    if heuristic == 1:
        h1 = heuristicOne(Root)
        Root.setHeuristic(h1)
        Root.setF(0,h1)
        
    elif heuristic == 2:
        h2 = heuristicTwo(Root)
        Root.setHeuristic(h2)
        Root.setF(0,h2)
    elif heuristic ==3:
        h3 = heuristicThree(Root)
        Root.setHeuristic(h3)
        Root.setF(0,h3)
#     elif heuristic ==4:
#         Root.setHeuristic(heuristicFour(Root))
    else:
#         print("Heuristic does not exist. Try again with valid heuristic.")
        return "Heuristic does not exist. Try again with valid heuristic."
        
        
    
    # Append the rootState (initial state) to the OPEN list
    OPEN.append(Root)
    
    # Check if the rootState is goal
    if Root.isGoal():
        OPEN.pop(0)
        CLOSED.append(Root)
        return CLOSED, getSolutionPath(CLOSED, Root)
    
    while True:
        
        # If the OPEN list is empty, then there is no solution found
        if not OPEN:
#             print("No Solution found.")
            return CLOSED, getSolutionPath(CLOSED, Root)
        
        
        
        # "Visit" the first board in the OPEN list and add it to the CLOSED list
        parentBoard = OPEN.pop(0)
        CLOSED.append(parentBoard)

        
        # Checking if the current board we are visiting is the goal state
        if  parentBoard.isGoal():
            # AFTER FINDING THE GOAL STATE WE HAVE TO BACKTRACK AND FIND SOLUTION PATH
            
            # CURRENTLY ONLY TEMPORARILY PRINTING OPEN CLOSED LISTS
#             print("CLOSED List: ")
#             for board in CLOSED:
#                 print(board.puzzleLine)

            
            return CLOSED, getSolutionPath(CLOSED, Root)
        else: 
            parentBoard.leaveGridIfAtExit()
            
            
            
        # Generate possible successor moves for the cars on the board
        successors = parentBoard.generateSuccessorBoards()
        
        for child in successors:
            
            
            # set heuristic
            if heuristic == 1:
                h1 = heuristicOne(child)
                child.setHeuristic(h1)
                child.setF(0,h1)

            elif heuristic == 2:
                h2 = heuristicTwo(child)
                child.setHeuristic(h2)
                child.setF(0,h2)
            elif heuristic ==3:
                h3 = heuristicThree(child)
                child.setHeuristic(h3)
                child.setF(0,h3)
#             elif heuristic ==4:
#                 h3 = heuristicFour(child)
#                 child.setHeuristic(h4)
#                 child.setF(0,h4)     
                
               
            # Checking if the current successor board is already present in the CLOSED list
            if(not any(child.grid == visitedBoard.grid for visitedBoard in CLOSED)):
                # If the OPEN list is empty, we will append the first child
                if not OPEN:
                    OPEN.append(child)
                else:    
                    # Checking if the current successor board is already present in the OPEN list
                    # If it is and the cost of its "clone" is greater, then we will replace it with the current successor
                    if any((child.grid == toBeVisited.grid and child.heuristic < toBeVisited.heuristic) for toBeVisited in OPEN):
                        OPEN.remove(toBeVisited)
                        OPEN.append(child)
                     # If it is a new successor, we append it to the list   
                    elif any((child.grid == toBeVisited.grid) for toBeVisited in OPEN):
                        continue
                    else:
                        OPEN.append(child)
        
        # Sort this list by cost (priority by cost)
        OPEN.sort(key=lambda x: x.heuristic, reverse=False)


In [9]:
testList = []
if any((1 == i )for i in testList):
    testList.remove(i)

In [10]:
def AlgorithmA(Root, heuristic):
    CLOSED = []
    OPEN = []
    
    
    
    if heuristic == 1:
        h1 = heuristicOne(Root)
        Root.setHeuristic(h1)
        Root.setF(Root.getCost(),h1)
        
    elif heuristic == 2:
        h2 = heuristicTwo(Root)
        Root.setHeuristic(h2)
        Root.setF(Root.getCost(),h2)
    elif heuristic ==3:
        h3 = heuristicThree(Root)
        Root.setHeuristic(h3)
        Root.setF(Root.getCost(),h3)
#     elif heuristic ==4:
#         Root.setHeuristic(heuristicFour(Root))
    else:
#         print("Heuristic does not exist. Try again with valid heuristic.")
        return "Heuristic does not exist. Try again with valid heuristic."
        
        
    
    # Append the rootState (initial state) to the OPEN list
    OPEN.append(Root)
    
    # Check if the rootState is goal
    if Root.isGoal():
        OPEN.pop(0)
        CLOSED.append(Root)
        return CLOSED, getSolutionPath(CLOSED, Root)
    
    while True:
        
        # If the OPEN list is empty, then there is no solution found
        if not OPEN:
#             print("No Solution found.")
            CLOSED, getSolutionPath(CLOSED, Root)
        
        
        
        # "Visit" the first board in the OPEN list and add it to the CLOSED list
        parentBoard = OPEN.pop(0)
        CLOSED.append(parentBoard)

        
        # Checking if the current board we are visiting is the goal state
        if  parentBoard.isGoal():
            # AFTER FINDING THE GOAL STATE WE HAVE TO BACKTRACK AND FIND SOLUTION PATH
            
            # CURRENTLY ONLY TEMPORARILY PRINTING OPEN CLOSED LISTS
#             print("CLOSED List: ")
#             for board in CLOSED:
#                 print(board.getF()," ",board.getCost()," ",board.getHeuristic()," ", end =" ")
#                 print(board.puzzleLine)

            
            return CLOSED, getSolutionPath(CLOSED, Root)
        else: 
            parentBoard.leaveGridIfAtExit()
            
            
            
        # Generate possible successor moves for the cars on the board
        successors = parentBoard.generateSuccessorBoards()
        
        for child in successors:
            
            
            # set heuristic
            if heuristic == 1:
                h1 = heuristicOne(child)
                child.setHeuristic(h1)
                child.setF(child.getCost(),h1)

            elif heuristic == 2:
                h2 = heuristicTwo(child)
                child.setHeuristic(h2)
                child.setF(child.getCost(),h2)
            elif heuristic ==3:
                h3 = heuristicThree(child)
                child.setHeuristic(h3)
                child.setF(child.getCost(),h3)
#             elif heuristic ==4:
#                 h3 = heuristicFour(child)
#                 child.setHeuristic(h4)
#                 child.setF(0,h4)     
                
               
            # Checking if the current successor board is already present in the CLOSED list
            if(not any(child.grid == visitedBoard.grid for visitedBoard in CLOSED)):
                # If the OPEN list is empty, we will append the first child
                if not OPEN:
                    OPEN.append(child)
                else:    
                    # Checking if the current successor board is already present in the OPEN list
                    # If it is and the cost of its "clone" is greater, then we will replace it with the current successor
                    if any((child.grid == toBeVisited.grid and child.f < toBeVisited.f) for toBeVisited in OPEN):
                        OPEN.remove(toBeVisited)
                        OPEN.append(child)
                     # If it is a new successor, we append it to the list   
                    elif any((child.grid == toBeVisited.grid) for toBeVisited in OPEN):
                        continue
                    else:
                        OPEN.append(child)
        
        # Sort this list by cost (priority by cost)
        OPEN.sort(key=lambda x: x.f, reverse=False)


## Heuristics

In [11]:
def heuristicOne(board):
    carsOnRight = []
    
    Ambulance = board.cars['A']
    rightOfA = Ambulance.coordinates[-1]  # right most coordiante of Ambulance [x,y]
    i = rightOfA[1]
        
    while(i < 5): # until we reach right edge of Grid # y coordinate
        i += 1 # check one cell to right
        cellContent = board.grid[rightOfA[0]][i]  # 'M' or '.' ..
        
        if cellContent == '.':
            continue
        else:
            if cellContent not in carsOnRight:
                carsOnRight.append(cellContent) # append the name
    
    return len(carsOnRight)
            



In [12]:
def heuristicTwo(board):
    carsOnRight = []
    
    Ambulance = board.cars['A']
    rightOfA = Ambulance.coordinates[-1]  # right most coordiante of Ambulance [x,y]
    i = rightOfA[1]
        
    while(i < 5): # until we reach right edge of Grid # y coordinate
        i += 1 # check one cell to right
        cellContent = board.grid[rightOfA[0]][i]  # 'M' or '.' ..
        
        if cellContent == '.':
            continue
        else:
            carsOnRight.append(cellContent) # append the name
    
    return len(carsOnRight)
            



In [13]:
def heuristicThree(board):
    alpha = 5
    h1 = heuristicOne(board)
    return h1*alpha

In [14]:
def heuristicFour(board):
    h1 = heuristicOne(board)
    return h1+1

## Test GBFS

In [15]:

lisa = Board("BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL.")
print("[TEST GBFS]")
start_time = time.time()
solutionPathLisa = GreedyBestFirstSearch(lisa, 1)
print("Runtime: ", time.time() - start_time,"\n")
print(solutionPathLisa)



[TEST GBFS]
Runtime:  0.007952451705932617 

([<__main__.Board object at 0x000002943B359F00>, <__main__.Board object at 0x000002943B35A7A0>, <__main__.Board object at 0x000002943B358670>, <__main__.Board object at 0x000002943B35ACE0>], ['M down 1    99 BBIJ....IJCC..IAA.GDDK.MGH.KLMGHFFL. M99', 'A right 1    99 BBIJ....IJCC..I.AAGDDK.MGH.KLMGHFFL. A99 M99', 'A right 1    99; M down 1    99; '])


## Test algorithm A

In [16]:
dk = Board("BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL.")

print("[TEST Algo A]")
start_time = time.time()
CLOSED, solutionPathDk= AlgorithmA(dk, 1)
print("Runtime: ", time.time() - start_time,"\n")
print(solutionPathDk)





# dk.printPuzzleLine()

# print("\ncar fuel available:")
# for each in dk.cars:
#     print(each+":"+str(dk.cars[each].fuel)+",", end = ' ')
        
        
# print("\nRuntime: ", round(time.time() - start_time,3)," seconds\n")
# print("search path length: ")
# print("solution path length: "+str(len(solutionPathDk))+" moves")
# print("solution path: "+solutionPathDk[-1])

# for each in solutionPathDk[:-1]:
#     print(each)

    
    
# print("\n! "+CLOSED[-1].puzzleLine[37:])
# CLOSED[-1].printPuzzleLine()








[TEST Algo A]
Runtime:  0.016059398651123047 

['M down 1    99 BBIJ....IJCC..IAA.GDDK.MGH.KLMGHFFL. M99', 'A right 1    99 BBIJ....IJCC..I.AAGDDK.MGH.KLMGHFFL. A99 M99', 'A right 1    99; M down 1    99; ']


## Save ourput files

In [17]:
def saveSolutionFile(fileName, board,runTime, solutionPath,CLOSED):
    
    
    try: 
        with open("output/"+fileName+".txt", "a") as f:
            
            print(board.returnPuzzleLine(),file=f)

            print("\ncar fuel available:",file=f)
            for each in board.cars:
                print(each+":"+str(board.cars[each].fuel)+",", end = ' ',file=f)

            if not solutionPath:
                print("\nRuntime: ", round(runTime,3)," seconds\n",file=f)
                print("search path length: ",len(CLOSED),file=f)
                print("solution path length: "+str(len(solutionPath))+" moves",file=f)
                print("solution path: "+solutionPath[-1],file=f)
                print("\n",file=f)

                for each in solutionPath[:-1]:
                    print(each,file=f)



                print("\n! "+CLOSED[-1].puzzleLine[37:],file=f)
                print(CLOSED[-1].returnPuzzleLine(),file=f)
            else:
                print("Sorry, could not solve the puzzle as specified.\nError: no solution found")
                 
    except IOError:
        print("Error: ")
        return 0
    finally:
        f.close()
    
    




In [18]:
def saveSearchFile(fileName,CLOSE):
    

    
    try: 
        with open("output/"+fileName+".txt", "a") as f:
             for board in CLOSED:
                if fileName[0] == "u":
                    print(board.getCost()," ",board.getCost()," 0 ", end =" ", file=f)
                elif fileName[0] == "g":    
                    print(board.getHeuristic()," 0 ",board.getHeuristic()," ", end =" ", file=f)
                elif fileName[0] == "a":    
                    print(board.getF()," ",board.getCost()," ",board.getHeuristic()," ", end =" ", file=f)
                print(board.puzzleLine, file=f)

                
                
    except IOError:
        print("Error: ")
        return 0
    finally:
        f.close()
    
    

In [19]:
# saveSearchFile("a-h1-search-1",CLOSED)

### Game engine

In [20]:
def gameEngine(file_path):
    i=1
    puzzles = readPuzzles(file_path)

    for each in puzzles:
        myBoard = Board(each)
        
        # ===UCS===
        start_time = time.time()
        uCLOSED, uSolutionPath = UniformCostSearch(myBoard)
        runtime = time.time() - start_time

        # save output files
        uSearchFileName = "ucs-search-"+str(i)
        uSolutionFileName = "ucs-sol-"+str(i)
        saveSearchFile(uSearchFileName, uCLOSED)
        saveSolutionFile(uSolutionFileName, myBoard,runtime, uSolutionPath,uCLOSED)
        
        
        for j in range(1,4): # TODO need to change (1,5) after adding h4 
        
            # ===GBFS===
            start_time = time.time()
            gCLOSED, gSolutionPath = GreedyBestFirstSearch(myBoard, j)
            runtime = time.time() - start_time


            # save output files
            g1SearchFileName = "gbfs-h"+str(j)+"-search-"+str(i)
            g1SolutionFileName = "gbfs-h"+str(j)+"-sol-"+str(i)
            saveSearchFile(gSearchFileName, g1CLOSED)
            saveSolutionFile(gSolutionFileName, myBoard,runtime, gSolutionPath,gCLOSED)


            # ===Algo A ===
            start_time = time.time()
            aCLOSED, aSolutionPath = UniformCostSearch(myBoard)
            runtime = time.time() - start_time

            # save output files
            aSearchFileName = "a-h"+str(j)+"-search-"+str(i)
            aSolutionFileName = "a-h"+str(j)+"-sol-"+str(i)
            saveSearchFile(aSearchFileName, uCLOSED)
            saveSolutionFile(aSolutionFileName, myBoard,runtime, aSolutionPath,aCLOSED)

        
        i +=1
        
        
        
        
        
        
        

## Run with given input file

In [21]:
def gameEngineExcel(file_path):

    puzzleNum= []
    algo = []
    heuristic = []
    lenSolutionPath= []
    lenClose = []
    runTime = []
    
    
    i=1
    puzzles = readPuzzles(file_path)

    for each in puzzles:
        myBoard = Board(each)
        
        # ===UCS===
        start_time = time.time()
        uCLOSED, uSolutionPath = UniformCostSearch(myBoard)
        runtime = round(time.time() - start_time,2)
        
        
        puzzleNum.append(i)
        algo.append("USC")
        heuristic.append("NA")
        lenSolutionPath.append(len(uSolutionPath[:-1]))
        lenClose.append(len(uCLOSED))
        runTime.append(runtime)
        

        
        for j in range(1,4): # TODO need to change (1,5) after adding h4 
        
            # ===GBFS ===
            start_time = time.time()
            gCLOSED, gSolutionPath = GreedyBestFirstSearch(myBoard, j)
            runtime = round(time.time() - start_time,2)
            

            puzzleNum.append(i)
            algo.append("GBFS")
            heuristic.append("h"+str(j))
            lenSolutionPath.append(len(gSolutionPath[:-1]))
            lenClose.append(len(gCLOSED))
            runTime.append(runtime,)


            # ===Algo A ===
            start_time = time.time()
            aCLOSED, aSolutionPath = UniformCostSearch(myBoard)
            runtime = round(time.time() - start_time,2)
            
            
            puzzleNum.append(i)
            algo.append("A/A*")
            heuristic.append("h"+str(j))
            lenSolutionPath.append(len(aSolutionPath[:-1]))
            lenClose.append(len(aCLOSED))
            runTime.append(runtime)
        
        i +=1
    


    df = DataFrame({'Puzzle Number': puzzleNum, 'Algorithm': algo, 'Heuristic': heuristic ,'Length of the Solution': lenSolutionPath, 'Length of the Search Path': lenClose,'ExecutionTime (in seconds)': runTime })
    df.to_excel('test.xlsx', index=False)
    

    

In [ ]:
gameEngineExcel("ourInput.txt")